In [56]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.base import BaseEstimator, ClassifierMixin
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Normal
import os, sys
sys.path.append(os.path.join("/home/ra43rid/torch_plnet"))
torch.set_default_device("cuda")
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class TorchMultivariateGaussianClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, class_params=None, device="cpu"):
        """
        Initialize the classifier with class parameters.

        Parameters:
        class_params: dict
            A dictionary where keys are class labels and values are dictionaries with
            'mean' (vector), 'cov' (matrix), and 'prior' for each class.
        device: str
            The device to use for computations ('cpu' or 'cuda').
        """
        self.class_params = class_params if class_params is not None else {}
        self.device = torch.device(device)
        self.classes_ = torch.tensor(list(self.class_params.keys()), device=self.device)
        self.n_features = len(self.class_params[0]["cov"])
        self.n_classes = len(self.classes_)
    
    def fit(self, X, y=None):
        """
        Fit method for compatibility. This classifier doesn't require fitting.
        """
        self.classes_ = torch.tensor(list(self.class_params.keys()), device=self.device)
        return self
    
    def predict_proba(self, X):
        """
        Predict the probability of each class for the given input data X.

        Parameters:
        X: torch.Tensor or array-like of shape (n_samples, n_features)
            Input features.

        Returns:
        probs: torch.Tensor of shape (n_samples, n_classes)
            Predicted probabilities for each class.
        """
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, device=self.device, dtype=torch.float32)
        
        probs = torch.zeros((X.shape[0], len(self.classes_)), device=self.device)
        for i, c in enumerate(self.classes_):
            mean = torch.tensor(self.class_params[int(c)]["mean"], device=self.device, dtype=torch.float32)
            cov = torch.tensor(self.class_params[int(c)]["cov"], device=self.device, dtype=torch.float32)
            prior = self.class_params[int(c)]["prior"]
            
            # Multivariate normal distribution
            mvn_dist = MultivariateNormal(mean, covariance_matrix=cov)
            px_given_y = torch.exp(mvn_dist.log_prob(X))
            
            # Combine with prior
            probs[:, i] = px_given_y * prior
        
        # Normalize to get P(y=c|x)
        probs /= probs.sum(dim=1, keepdim=True)
        return probs
    
    def predict(self, X):
        """
        Predict the class label for each sample in X.

        Parameters:
        X: torch.Tensor or array-like of shape (n_samples, n_features)
            Input features.

        Returns:
        predictions: torch.Tensor of shape (n_samples,)
            Predicted class labels.
        """
        probs = self.predict_proba(X)
        return self.classes_[torch.argmax(probs, dim=1)]
    
    def generate_data(self, n_samples=100, seed=42):
        """
        Generate synthetic data using the predefined class parameters.

        Parameters:
        n_samples: int
            Number of samples to generate.

        Returns:
        X: torch.Tensor of shape (n_samples, n_features)
            Generated features.
        y: torch.Tensor of shape (n_samples,)
            Generated labels.
        """
        X = []
        y = []

        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        for _ in range(n_samples):
            # Sample class based on priors
            sampled_class = torch.multinomial(
                torch.tensor([self.class_params[int(c)]["prior"] for c in self.classes_], device=self.device),
                num_samples=1
            ).item()
            mean = torch.tensor(self.class_params[int(self.classes_[sampled_class])]["mean"], device=self.device, dtype=torch.float32)
            cov = torch.tensor(self.class_params[int(self.classes_[sampled_class])]["cov"], device=self.device, dtype=torch.float32)
            
            # Sample feature vector from the corresponding multivariate Gaussian
            mvn_dist = MultivariateNormal(mean, covariance_matrix=cov)
            sampled_x = mvn_dist.sample()
            X.append(sampled_x)
            y.append(self.classes_[sampled_class].item())
        
        return torch.stack(X), torch.tensor(y, device=self.device)

class TorchGaussianSyntheticClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, class_params=None, device="cpu"):
        """
        Initialize the classifier with class parameters.

        Parameters:
        class_params: dict
            A dictionary where keys are class labels and values are dictionaries with
            'mean', 'std', and 'prior' for each class.
        device: str
            The device to use for computations ('cpu' or 'cuda').
        """
        self.class_params = class_params if class_params is not None else {}
        self.device = torch.device(device)
        self.classes_ = torch.tensor(list(self.class_params.keys()), device=self.device)
        self.n_features = 1
        self.n_classes = len(self.classes_)
    
    def fit(self, X, y=None):
        """
        Fit method for compatibility. This classifier doesn't require fitting.
        """
        self.classes_ = torch.tensor(list(self.class_params.keys()), device=self.device)
        return self
    
    def predict_proba(self, X):
        """
        Predict the probability of each class for the given input data X.

        Parameters:
        X: torch.Tensor or array-like of shape (n_samples,)
            Input features.

        Returns:
        probs: torch.Tensor of shape (n_samples, n_classes)
            Predicted probabilities for each class.
        """
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, device=self.device, dtype=torch.float32)
        
        probs = torch.zeros((len(X), len(self.classes_)), device=self.device)
        for i, c in enumerate(self.classes_):
            mean = self.class_params[int(c)]["mean"]
            std = self.class_params[int(c)]["std"]
            prior = self.class_params[int(c)]["prior"]
            
            # Calculate Gaussian PDF: P(x|y=c)
            normal_dist = Normal(loc=mean, scale=std)
            px_given_y = torch.exp(normal_dist.log_prob(X))
            
            # Combine with prior: P(x|y=c) * P(y=c)
            probs[:, i] = px_given_y * prior
        
        # Normalize to get P(y=c|x)
        probs /= probs.sum(dim=1, keepdim=True)
        return probs
    
    def predict(self, X):
        """
        Predict the class label for each sample in X.

        Parameters:
        X: torch.Tensor or array-like of shape (n_samples,)
            Input features.

        Returns:
        predictions: torch.Tensor of shape (n_samples,)
            Predicted class labels.
        """
        probs = self.predict_proba(X)
        return self.classes_[torch.argmax(probs, dim=1)]

    def generate_data(self, n_samples=100, seed=42):
        """
        Generate synthetic data using the predefined class parameters.

        Parameters:
        n_samples: int
            Number of samples to generate.

        Returns:
        X: torch.Tensor of shape (n_samples,)
            Generated features.
        y: torch.Tensor of shape (n_samples,)
            Generated labels.
        """


        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


        X = []
        y = []
        for _ in range(n_samples):
            # Sample class based on priors
            sampled_class = torch.multinomial(
                torch.tensor([self.class_params[int(c)]["prior"] for c in self.classes_], device=self.device),
                num_samples=1
            ).item()
            # Sample feature value from the corresponding Gaussian
            mean = self.class_params[int(self.classes_[sampled_class])]["mean"]
            std = self.class_params[int(self.classes_[sampled_class])]["std"]
            normal_dist = Normal(loc=mean, scale=std)
            sampled_x = normal_dist.sample().item()
            X.append(sampled_x)
            y.append(self.classes_[sampled_class].item())
        
        return torch.tensor(X, device=self.device), torch.tensor(y, device=self.device)

def goodman_kruskal_gamma(x, y):
    """
    Compute Goodman and Kruskal's Gamma for two ordinal variables.
    
    Parameters:
    x, y: Lists or arrays of ordinal data (same length)
    
    Returns:
    gamma: Goodman and Kruskal's Gamma
    """
    if len(x) != len(y):
        raise ValueError("Both variables must have the same length.")
    
    concordant = 0
    discordant = 0
    
    n = len(x)
    for i in range(n - 1):
        for j in range(i + 1, n):
            # Determine concordance or discordance
            if (x[i] > x[j] and y[i] > y[j]) or (x[i] < x[j] and y[i] < y[j]):
                concordant += 1
            elif (x[i] > x[j] and y[i] < y[j]) or (x[i] < x[j] and y[i] > y[j]):
                discordant += 1
    
    # Compute Gamma
    if concordant + discordant == 0:
        return 0  # Avoid division by zero
    gamma = (concordant - discordant) / (concordant + discordant)
    return gamma

from torchcp.classification.score import APS, THR, SAPS
aps = APS(score_type="identity", randomized=False)
rand_aps = APS(score_type="identity", randomized=True)
lac = THR(score_type="identity",)
saps = SAPS(score_type="identity",randomized=False)

class_params_1f_2c = {
    0: {"mean": 1, "std": 1, "prior": 0.3},
    1: {"mean": 3, "std": 1, "prior": 0.4},
    # 2: {"mean": 4, "std": 2.2, "prior": 0.3},
}   

class_params_1f_3c = {
    0: {"mean": 1, "std": 1, "prior": 0.3},
    1: {"mean": 3, "std": 1, "prior": 0.4},
    2: {"mean": 4, "std": 2.2, "prior": 0.3},
}   

# Initialize and fit the generator
class_params_2d_3c = {
    0: {
        "mean": [2.5, 2.0],  # Mean vector for class 0
        "cov": [
            [1.0, 0.5],
            [0.5, 1.2],
            # [0.3, 0.4, 0.8]
        ],  # Covariance matrix for class 0
        "prior": 0.3  # Prior probability for class 0
    },
    1: {
        "mean": [3.5, 4.0],  # Mean vector for class 1
        "cov": [
            [1.5, 0.3],
            [0.3, 1.1],
            # [0.2, 0.1, 0.9]
        ],  # Covariance matrix for class 1
        "prior": 0.4  # Prior probability for class 1
    },
    2: {
        "mean": [2.0, 1.0],  # Mean vector for class 2
        "cov": [
            [1.2, 0.4],
            [0.4, 1.3],
            # [0.3, 0.5, 1.4]
        ],  # Covariance matrix for class 2
        "prior": 0.3  # Prior probability for class 2
    },
}

# Initialize and fit the generator
class_params_3d_3c = {
    0: {
        "mean": [2.0, 2.0, 1.0],  # Mean vector for class 0
        "cov": [
            [1.0, 0.5, 0.3],
            [0.5, 1.2, 0.4],
            [0.3, 0.4, 0.8]
        ],  # Covariance matrix for class 0
        "prior": 1/3  # Prior probability for class 0
    },
    1: {
        "mean": [3.0, 4.0, 3.0],  # Mean vector for class 1
        "cov": [
            [1.5, 0.3, 0.2],
            [0.3, 1.1, 0.1],
            [0.2, 0.1, 0.9]
        ],  # Covariance matrix for class 1
        "prior": 1/3  # Prior probability for class 1
    },
    2: {
        "mean": [1.0, 2.0, 4.0],  # Mean vector for class 2
        "cov": [
            [1.2, 0.4, 0.3],
            [0.4, 1.3, 0.5],
            [0.3, 0.5, 1.4]
        ],  # Covariance matrix for class 2
        "prior": 1/3  # Prior probability for class 2
    },
}


# Initialize and fit the generator
class_params_2d_2c = {
    0: {
        "mean": [3.0, 2.0],  # Mean vector for class 0
        "cov": [
            [1.0, 0.5],
            [0.5, 1.2],
        ],  # Covariance matrix for class 0
        "prior": 0.3  # Prior probability for class 0
    },
    1: {
        "mean": [2.0, 3.0],  # Mean vector for class 1
        "cov": [
            [1.5, 0.3],
            [0.3, 1.1],
        ],  # Covariance matrix for class 1
        "prior": 0.4  # Prior probability for class 1
    },
    # 2: {
    #     "mean": [1.0, 2.0, 2.0],  # Mean vector for class 2
    #     "cov": [
    #         [1.2, 0.4],
    #         [0.4, 1.3],
    #         # [0.3, 0.5, 1.4]
    #     ],  # Covariance matrix for class 2
    #     "prior": 0.3  # Prior probability for class 2
    # },
}

gen_1d_2c = TorchGaussianSyntheticClassifier(class_params=class_params_1f_2c, device="cuda")
gen_1d_3c = TorchGaussianSyntheticClassifier(class_params=class_params_1f_3c, device="cuda")
gen_3d_3c = TorchMultivariateGaussianClassifier(class_params=class_params_3d_3c, device="cuda")
gen_2d_2c = TorchMultivariateGaussianClassifier(class_params=class_params_2d_2c, device="cuda")
gen_2d_3c = TorchMultivariateGaussianClassifier(class_params=class_params_2d_3c, device="cuda")

In [57]:
class OracleAnnotator:
    def __init__(self,score, generator):
        self.score = score
        self.classes_ = generator.classes_
        self.generator = generator

    # we assume y is already label encoded
    def get_conformity(self, X, y):
        y_pred_proba = self.generator.predict_proba(X)
        scores = self.score(y_pred_proba, y)
        return scores

In [58]:
from zmq import device
from util.ranking_datasets import LabelPairDataset
from models.ranking_models import LabelRankingModel
from torch.utils.data.dataloader import DataLoader
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
import torch
from scipy.stats import kendalltau
from joblib import Parallel, delayed
torch.set_default_device("cuda")
import hashlib

def derive_seed(seed, index):
    combined = f"{seed}-{index}".encode()
    return int(hashlib.sha256(combined).hexdigest(), 16) % (2**32)



In [59]:
gen = gen_3d_3c
seeds = list(range(0,5))


In [60]:
def create_from_classification_data(X, y, num_classes):
    X_pairs = []
    y_pairs = []

    for X_vec, label in zip(X, y):
        for k in range(0, num_classes):
            if k == label:
                continue
            X_pairs.append(
                torch.vstack(
                    [
                        torch.tensor(X_vec, dtype=torch.float32),
                        torch.tensor(X_vec, dtype=torch.float32),
                    ]
                )
            )
            y_pairs.append(
                torch.vstack(
                    [
                        torch.tensor(label, dtype=torch.long),
                        torch.tensor(k, dtype=torch.long),
                    ]
                )
            )

    X_pairs = torch.stack(X_pairs)
    y_pairs = torch.stack(y_pairs)
    return X_pairs, y_pairs


def generate_bt_preferences_batch(gen, X,y, use_bt = True):

    probs = gen.predict_proba(X)
    prob_tensor = torch.take_along_dim(probs, y.unsqueeze(-1), dim=1).squeeze()
    assert prob_tensor.dim() == 1 and prob_tensor.size(0) % 2 == 0, "Input must be 1D tensor of even length"



    pairs = prob_tensor.view(-1, 2)
    s1 = pairs[:, 0]
    s2 = pairs[:, 1]

    if use_bt:
        prob_1_pref = torch.sigmoid(s1 - s2)
        preferences = torch.bernoulli(prob_1_pref)
    else:
        preferences = s1 > s2
    
    X_pairs = X.view(-1, 2, X.size(1))      # (N/2, 2, D)
    y_pairs = y.view(-1, 2)                 # (N/2, 2)
    prob_pairs = probs.view(-1, 2)          # (N/2, 2)

    # Bradley-Terry preference: probability first is preferred
    prob_1_pref = torch.sigmoid(prob_pairs[:, 0] - prob_pairs[:, 1])
    # Reorder both features and labels

    preferred_X = torch.where(preferences.unsqueeze(-1), X_pairs[:, 0], X_pairs[:, 1])
    nonpreferred_X = torch.where(preferences.unsqueeze(-1), X_pairs[:, 1], X_pairs[:, 0])
    paired_X = torch.stack([preferred_X, nonpreferred_X], dim=1)  # (N/2, 2, D)

    preferred_y = torch.where(preferences, y_pairs[:, 0], y_pairs[:, 1])
    nonpreferred_y = torch.where(preferences, y_pairs[:, 1], y_pairs[:, 0])
    paired_y = torch.stack([preferred_y, nonpreferred_y], dim=1)  # (N/2, 2)
    return paired_X, paired_y.unsqueeze(-1)

In [61]:
from models.classifier_model import ClassifierModel
from util.ranking_datasets import TabularDataset
def fit_classifier(X_train, y_train, X_es, y_es):

    model = ClassifierModel(
        input_dim=X_train.shape[1], hidden_dims=[5], output_dim=3
    )
    # model.cuda()
    # model.fit(
    #     X_train.cpu().detach().numpy(),
    #     y_train.cpu().detach().numpy(),
    #     num_epochs=500,
    #     patience=16,
    #     batch_size=32,
    #     val_frac=0.0,
    #     learning_rate=0.01,
    # )

    ds = TabularDataset(X_train, y_train)
    train_loader = DataLoader(ds, batch_size=32)
    ds_es = TabularDataset(X_es, y_es)
    es_loader = DataLoader(ds_es, batch_size=32)
    model._fit(
        train_loader=train_loader,
        val_loader=es_loader,
        num_epochs=500,
        patience=16,
        learning_rate=0.01,
        # verbose=True
    )


    return model

In [62]:
from models.ranking_models import LabelRankingModel
def fit_ranker(X_train, y_train, X_pairs=None, y_pairs=None, X_es = None, y_es=None):

    model = LabelRankingModel(
        input_dim=X_train.shape[1], hidden_dims=[5], output_dim=3
    )
    model.cuda()

    X_train_prf, y_train_prf = create_from_classification_data(X_train, y_train, 3)

    X_train_overall = torch.concat([X_train_prf, X_pairs])
    y_train_overall = torch.concat([y_train_prf, y_pairs])


    print(len(y_train_prf),len(y_train), len(y_train_overall))
    ds = LabelPairDataset()
    ds.create_from_numpy_pairs(X_train_overall, y_train_overall)
    pair_loader = DataLoader(ds, batch_size=32)
    X_es_pairs, y_es_pairs = create_from_classification_data(X_es, y_es, 3)
    ds_es = LabelPairDataset()
    ds_es.create_from_numpy_pairs(X_es_pairs, y_es_pairs)

    es_loader = DataLoader(ds_es, batch_size=32)
    model._fit(
        train_loader=pair_loader,
        val_loader=es_loader,
        num_epochs=500,
        patience=16,
        learning_rate=0.01,
        verbose=True
    )

    return model

In [ ]:
from itertools import product
from sklearn.metrics import accuracy_score
from torchcp.classification.score import APS, THR, TOPK, RAPS, SAPS, Margin, KNN
from conformal.conformal import IDENTITY, OWN_APS
from torchcp.classification.predictor import SplitPredictor
from torchcp.classification import Metrics
from itertools import product
from util.ranking_datasets import TabularDataset, LabelPairDataset


nums_clf_data = [10,20,30,40,50,100,150]
# nums_clf_data = [25,50,75,100]
master_seeds = list(range(20))
num_prf_data = 100
num_test_data = 100
num_calib_data = 100
num_es_data = 100

results = []

for master_seed, num_clf_data in product(master_seeds, nums_clf_data):

    X_calib, y_calib = gen.generate_data(num_calib_data, seed=derive_seed(master_seed,1))
    # two times the number because we want num_prf_data preferences
    X_prf, y_prf = gen.generate_data(2*num_prf_data, seed=derive_seed(master_seed,2))
    X_pairs, y_pairs = generate_bt_preferences_batch(gen, X_prf, y_prf, False)

    X_clf, y_clf = gen.generate_data(num_clf_data, seed=derive_seed(master_seed,3))

    X_test, y_test = gen.generate_data(num_test_data, seed=derive_seed(master_seed,4))

    X_es, y_es = gen.generate_data(num_es_data, seed=derive_seed(master_seed,5))

    ranker = fit_ranker(X_clf, y_clf, X_pairs, y_pairs, X_es, y_es)
    classifier = fit_classifier(X_clf, y_clf, X_es, y_es)

    ranker.num_classes = 3

    test_dataset = TabularDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32)

    # Classifier

    names = ["rand_aps", "aps", "thr"]
    alphas = [0.02,0.05,0.1,0.2]

    conformity_scores = [
        APS(randomized=True),
        APS(randomized=False),
        THR()
    ]
    for (name, conformity_score), alpha in product(
        zip(names, conformity_scores), alphas
    ):
        predictor = SplitPredictor(conformity_score, classifier)

        gradient_updates = predictor._model.gradient_updates

        cal_dataset = TabularDataset(X_calib, y_calib)

        cal_loader = DataLoader(cal_dataset,batch_size=32)

        predictor.calibrate(cal_loader, alpha)

        predictions_sets_list = []
        predictions_list = []
        labels_list = []
        logits_list = []
        feature_list = []

        # Evaluate in inference mode
        predictor._model.eval()
        with torch.no_grad():
            for batch in test_loader:
                # Move batch to device and get predictions
                inputs = batch[0]
                labels = batch[1]

                batch_predictions = predictor.predict(inputs)

                logits = classifier(inputs)

                predicted_label = logits.argmax(axis=1)
                # Accumulate predictions and labels
                predictions_sets_list.append(batch_predictions)
                predictions_list.append(predicted_label)
                labels_list.append(labels)
                logits_list.append(logits)
                feature_list.append(inputs)

            # Concatenate all batches
            val_prediction_sets = torch.cat(predictions_sets_list, dim=0)  # (N_val x C)
            val_predictions = torch.cat(predictions_list, dim=0)
            val_labels = torch.cat(labels_list, dim=0)  # (N_val,)
            val_logits = torch.cat(logits_list, dim=0)
            val_features = torch.cat(feature_list, dim=0)

            y_pred = val_predictions.detach().cpu().numpy()
            y_true = val_labels.detach().cpu().numpy()
            # Compute evaluation metrics
            metric = Metrics()
            avg_size = metric("average_size")(
                    prediction_sets=val_prediction_sets, labels=val_labels
                )
            coverage = metric("coverage_rate")(
                    prediction_sets=val_prediction_sets, labels=val_labels
                )
            accuracy = accuracy_score(y_true, y_pred)

            results.append([master_seed, num_clf_data, alpha, name, avg_size, coverage, accuracy])

    # Ranker
    test_dataset = TabularDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32)

    alphas = [0.02,0.05,0.1,0.2]

    for alpha in alphas:
        predictor = SplitPredictor(IDENTITY(), ranker)

        gradient_updates = predictor._model.gradient_updates

        cal_dataset = TabularDataset(X_calib, y_calib)

        cal_loader = DataLoader(cal_dataset,batch_size=32)

        predictor.calibrate(cal_loader, alpha)

        predictions_sets_list = []
        predictions_list = []
        labels_list = []
        logits_list = []
        feature_list = []

        # Evaluate in inference mode
        predictor._model.eval()
        with torch.no_grad():
            for batch in test_loader:
                # Move batch to device and get predictions
                inputs = batch[0]
                labels = batch[1]

                batch_predictions = predictor.predict(inputs)

                logits = classifier(inputs)

                predicted_label = logits.argmax(axis=1)
                # Accumulate predictions and labels
                predictions_sets_list.append(batch_predictions)
                predictions_list.append(predicted_label)
                labels_list.append(labels)
                logits_list.append(logits)
                feature_list.append(inputs)

            # Concatenate all batches
            val_prediction_sets = torch.cat(predictions_sets_list, dim=0)  # (N_val x C)
            val_predictions = torch.cat(predictions_list, dim=0)
            val_labels = torch.cat(labels_list, dim=0)  # (N_val,)
            val_logits = torch.cat(logits_list, dim=0)
            val_features = torch.cat(feature_list, dim=0)

            y_pred = val_predictions.detach().cpu().numpy()
            y_true = val_labels.detach().cpu().numpy()
            # Compute evaluation metrics
            metric = Metrics()

            avg_size = metric("average_size")(
                    prediction_sets=val_prediction_sets, labels=val_labels
                )
            coverage = metric("coverage_rate")(
                    prediction_sets=val_prediction_sets, labels=val_labels
                )
            accuracy = accuracy_score(y_true, y_pred)
            results.append([master_seed, num_clf_data, alpha, "ranker", avg_size, coverage, accuracy])


/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


20 10 120
Epoch 1/500
  Train Loss: 0.1937
  Val Loss: 0.0976
Epoch 2/500
  Train Loss: 0.1827
  Val Loss: 0.0961
Epoch 3/500
  Train Loss: 0.1748
  Val Loss: 0.0957
Epoch 4/500
  Train Loss: 0.1693
  Val Loss: 0.0959
Epoch 5/500
  Train Loss: 0.1654
  Val Loss: 0.0962
Epoch 6/500
  Train Loss: 0.1625
  Val Loss: 0.0966
Epoch 7/500
  Train Loss: 0.1602
  Val Loss: 0.0969
Epoch 8/500
  Train Loss: 0.1583
  Val Loss: 0.0970
Epoch 9/500
  Train Loss: 0.1567
  Val Loss: 0.0969
Epoch 10/500
  Train Loss: 0.1550
  Val Loss: 0.0968
Epoch 11/500
  Train Loss: 0.1534
  Val Loss: 0.0964
Epoch 12/500
  Train Loss: 0.1517
  Val Loss: 0.0959
Epoch 13/500
  Train Loss: 0.1498
  Val Loss: 0.0952
Epoch 14/500
  Train Loss: 0.1478
  Val Loss: 0.0943
Epoch 15/500
  Train Loss: 0.1457
  Val Loss: 0.0932
Epoch 16/500
  Train Loss: 0.1436
  Val Loss: 0.0919
Epoch 17/500
  Train Loss: 0.1415
  Val Loss: 0.0902
Epoch 18/500
  Train Loss: 0.1393
  Val Loss: 0.0883
Epoch 19/500
  Train Loss: 0.1370
  Val Loss:

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-pack

40 20 140
Epoch 1/500
  Train Loss: 0.1507
  Val Loss: 0.0969
Epoch 2/500
  Train Loss: 0.1426
  Val Loss: 0.0949
Epoch 3/500
  Train Loss: 0.1375
  Val Loss: 0.0941
Epoch 4/500
  Train Loss: 0.1342
  Val Loss: 0.0938
Epoch 5/500
  Train Loss: 0.1319
  Val Loss: 0.0936
Epoch 6/500
  Train Loss: 0.1302
  Val Loss: 0.0933
Epoch 7/500
  Train Loss: 0.1288
  Val Loss: 0.0929
Epoch 8/500
  Train Loss: 0.1275
  Val Loss: 0.0923
Epoch 9/500
  Train Loss: 0.1262
  Val Loss: 0.0916
Epoch 10/500
  Train Loss: 0.1247
  Val Loss: 0.0906
Epoch 11/500
  Train Loss: 0.1231
  Val Loss: 0.0894
Epoch 12/500
  Train Loss: 0.1213
  Val Loss: 0.0880
Epoch 13/500
  Train Loss: 0.1194
  Val Loss: 0.0865
Epoch 14/500
  Train Loss: 0.1175
  Val Loss: 0.0847
Epoch 15/500
  Train Loss: 0.1154
  Val Loss: 0.0827
Epoch 16/500
  Train Loss: 0.1133
  Val Loss: 0.0805
Epoch 17/500
  Train Loss: 0.1110
  Val Loss: 0.0782
Epoch 18/500
  Train Loss: 0.1088
  Val Loss: 0.0759
Epoch 19/500
  Train Loss: 0.1065
  Val Loss:

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-pack

...

20 10 120
Epoch 1/500
  Train Loss: 0.1817
  Val Loss: 0.1035
Epoch 2/500
  Train Loss: 0.1777
  Val Loss: 0.1012
Epoch 3/500
  Train Loss: 0.1751
  Val Loss: 0.0997
Epoch 4/500
  Train Loss: 0.1734
  Val Loss: 0.0986
Epoch 5/500
  Train Loss: 0.1722
  Val Loss: 0.0977
Epoch 6/500
  Train Loss: 0.1713
  Val Loss: 0.0970
Epoch 7/500
  Train Loss: 0.1705
  Val Loss: 0.0962
Epoch 8/500
  Train Loss: 0.1696
  Val Loss: 0.0953
Epoch 9/500
  Train Loss: 0.1686
  Val Loss: 0.0943
Epoch 10/500
  Train Loss: 0.1676
  Val Loss: 0.0933
Epoch 11/500
  Train Loss: 0.1665
  Val Loss: 0.0922
Epoch 12/500
  Train Loss: 0.1653
  Val Loss: 0.0912
Epoch 13/500
  Train Loss: 0.1640
  Val Loss: 0.0901
Epoch 14/500
  Train Loss: 0.1627
  Val Loss: 0.0890
Epoch 15/500
  Train Loss: 0.1613
  Val Loss: 0.0878
Epoch 16/500
  Train Loss: 0.1597
  Val Loss: 0.0865
Epoch 17/500
  Train Loss: 0.1581
  Val Loss: 0.0851
Epoch 18/500
  Train Loss: 0.1564
  Val Loss: 0.0836
Epoch 19/500
  Train Loss: 0.1546
  Val Loss:

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-pack

40 20 140
Epoch 1/500
  Train Loss: 0.1454
  Val Loss: 0.1040
Epoch 2/500
  Train Loss: 0.1417
  Val Loss: 0.1017
Epoch 3/500
  Train Loss: 0.1397
  Val Loss: 0.1002
Epoch 4/500
  Train Loss: 0.1385
  Val Loss: 0.0990
Epoch 5/500
  Train Loss: 0.1377
  Val Loss: 0.0978
Epoch 6/500
  Train Loss: 0.1369
  Val Loss: 0.0967
Epoch 7/500
  Train Loss: 0.1362
  Val Loss: 0.0956
Epoch 8/500
  Train Loss: 0.1354
  Val Loss: 0.0945
Epoch 9/500
  Train Loss: 0.1345
  Val Loss: 0.0934
Epoch 10/500
  Train Loss: 0.1335
  Val Loss: 0.0923
Epoch 11/500
  Train Loss: 0.1325
  Val Loss: 0.0912
Epoch 12/500
  Train Loss: 0.1314
  Val Loss: 0.0899
Epoch 13/500
  Train Loss: 0.1302
  Val Loss: 0.0885
Epoch 14/500
  Train Loss: 0.1289
  Val Loss: 0.0870
Epoch 15/500
  Train Loss: 0.1275
  Val Loss: 0.0854
Epoch 16/500
  Train Loss: 0.1260
  Val Loss: 0.0837
Epoch 17/500
  Train Loss: 0.1245
  Val Loss: 0.0819
Epoch 18/500
  Train Loss: 0.1229
  Val Loss: 0.0801
Epoch 19/500
  Train Loss: 0.1212
  Val Loss:

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/ra43rid/torch_plnet/venv/lib/python3.10/site-pack

60 30 160
Epoch 1/500
  Train Loss: 0.1466
  Val Loss: 0.1030
Epoch 2/500
  Train Loss: 0.1424
  Val Loss: 0.1005
Epoch 3/500
  Train Loss: 0.1400
  Val Loss: 0.0990
Epoch 4/500
  Train Loss: 0.1386
  Val Loss: 0.0979
Epoch 5/500
  Train Loss: 0.1376
  Val Loss: 0.0969
Epoch 6/500
  Train Loss: 0.1367
  Val Loss: 0.0958
Epoch 7/500
  Train Loss: 0.1357
  Val Loss: 0.0947
Epoch 8/500
  Train Loss: 0.1347
  Val Loss: 0.0935
Epoch 9/500
  Train Loss: 0.1335
  Val Loss: 0.0922
Epoch 10/500
  Train Loss: 0.1323
  Val Loss: 0.0909
Epoch 11/500
  Train Loss: 0.1310
  Val Loss: 0.0896
Epoch 12/500
  Train Loss: 0.1295
  Val Loss: 0.0881
Epoch 13/500
  Train Loss: 0.1280
  Val Loss: 0.0864


In [ ]:
results

[[0, 10, 0.02, 'rand_aps', 2.7200000286102295, 0.9900000095367432, 0.36],
 [0, 10, 0.05, 'rand_aps', 2.3299999237060547, 0.9700000286102295, 0.36],
 [0, 10, 0.1, 'rand_aps', 1.96999990940094, 0.9100000262260437, 0.36],
 [0, 10, 0.2, 'rand_aps', 1.7699999809265137, 0.8199999928474426, 0.36],
 [0, 10, 0.02, 'aps', 2.93999981880188, 0.9900000095367432, 0.36],
 [0, 10, 0.05, 'aps', 2.93999981880188, 0.9900000095367432, 0.36],
 [0, 10, 0.1, 'aps', 1.959999918937683, 0.8899999856948853, 0.36],
 [0, 10, 0.2, 'aps', 1.7899999618530273, 0.75, 0.36],
 [0, 10, 0.02, 'thr', 2.5999999046325684, 0.9900000095367432, 0.36],
 [0, 10, 0.05, 'thr', 2.129999876022339, 0.949999988079071, 0.36],
 [0, 10, 0.1, 'thr', 1.959999918937683, 0.9200000166893005, 0.36],
 [0, 10, 0.2, 'thr', 1.71999990940094, 0.8100000023841858, 0.36],
 [0, 10, 0.02, 'ranker', 2.2300000190734863, 1.0, 0.36],
 [0, 10, 0.05, 'ranker', 1.7999999523162842, 0.9800000190734863, 0.36],
 [0, 10, 0.1, 'ranker', 1.25, 0.9399999976158142, 0.36]

In [ ]:
import pandas as pd
result_df = pd.DataFrame(data=results, columns=["seed","num_clf_data","alpha","method","avg_set_size","coverage_rate","accuracy"])
result_df.to_csv("results_synthetic_sparse.csv")
# result_df = result_df[result_df.alpha==0.1]
result_df = result_df[result_df.method!="aps"]